In [1]:
%pylab inline
import pandas as pd
from dateparser import parse
from tqdm import tqdm

from sklearn.linear_model import LinearRegression

Populating the interactive namespace from numpy and matplotlib


In [2]:
ex = pd.ExcelFile('WDImetadata.xlsx')
explainer = ex.parse('2010-2019 MetaData').set_index('Code').fillna('')
def explain(c):
    global explainer
    return explainer.loc[c]['Long definition']

In [3]:
'''
Data Cleaning
'''

df = pd.read_excel('WDIdata3decades.xlsx')
df.iloc[:,2:] = df.iloc[:,2:].apply(pd.to_numeric, errors='coerce')

'''
Find codes that exist in all three decades:
'''
decades = ['1990-1999 MetaData',
           '2000-2010 MetaData',
           '2010-2019 MetaData']
decade_codes = [set(ex.parse(d)['Code']) for d in decades]

'''
Only include columns where data is preserved accross the decades Not_Null_Thresh times.
'''
decade_count = df.groupby('Custom_Time_Code').apply(lambda x: pd.notnull(x).sum(0))
not_null_thresh = 20
preserved = ((decade_count > not_null_thresh).sum() == decade_count.shape[0])
data = df.T[preserved].T

#Cast time code to datetime
data.Custom_Time_Code = data.Custom_Time_Code.apply(lambda x: parse(x[:4]))

In [20]:
def interp_w_nan(s, drop_idx = True, clip = True):
    if sum(pd.notnull(s))==0:
        return s
    else:
        old_idx = s.index
        if drop_idx:
            s = s.reset_index(drop=True)

        null_idxs = s.index[~pd.notnull(s)]

        x = np.array(s.dropna().index)[:, np.newaxis]
        y = s.dropna().astype(float).values
        reg = LinearRegression().fit(x, y)

        for i in null_idxs:
            pred = reg.predict(np.array([[i]]))[0]
            if pred < 0 and clip:
                pred = 0
            s.loc[i] = pred

        s.index = old_idx

        return s


In [22]:
filled = data.copy()
for col in tqdm(filled.columns[2:]):
    filled[col] = filled.groupby('Country Code')[col].apply(interp_w_nan).astype(float)

  2%|▏         | 17/938 [00:03<03:08,  4.89it/s]


KeyboardInterrupt: 

In [6]:
filled.to_csv('cleaned.csv')